# S2W6D4：工业级数据流水线（The Data Pipeline）

前几天我们都是在“手工作坊”里工作——手动定义一两个字符串，手动 Tokenize，手动喂给模型。但在真正的算法岗位（无论是 NLP 还是具身智能），我们要处理的数据量通常是 **百万级** 的。如果还用 `for` 循环一条条处理，你的训练代码可能要跑一万年。我们将使用 Hugging Face 生态中最高效的工具库 —— **`datasets`**。

**📅 日期**: Day 32
**🎯 核心目标**: 掌握处理 **海量数据** 的标准范式。
在 Day 1-3，我们是“手工作坊”，一次处理一句话。
在工业界（搜索/推荐/大模型训练），我们需要处理 **TB 级别** 的数据。如果还用 `for` 循环 + `Pandas`，内存早就爆了。今天我们要掌握 Hugging Face 的核武器 —— **`datasets` 库** 和 **`DataCollator`**。

## 1 🏭理论：为什么要用`datasets`库？

很多初学者喜欢用Pandas (`pd.read_csv`)。但在AI训练中，`datasets`库才是标准答案。

- **内存映射（Memory Mapping）**：
    - **问题**：你有100GB的语料，内存只有16GB，Pandas直接爆内存。
    - **解法**：`datasets`使用Apache Arrow格式，它把硬盘当内存用，读取1TB数据和1MB数据的初始开销几乎一样快。
- **Lazy Processing**：
    - 它不会一次性把所有数据都算完，而是等你用到那一条时才去加载（结合 Map 机制）。

## 2\. 💻 代码实战：构建流水线

### 2.1 准备数据集

为了模拟真实工作流，我们先在本地生成一个 CSV 文件（模拟你从公司数仓导出的数据）。

In [1]:
import os
import csv

# 1. 创建模拟数据 (通用文本分类场景：比如电商评论情感分析)
data_dir = "../../data/raw_data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

csv_path = os.path.join(data_dir, "comments.csv")

# 写入 5 条长短不一的数据
# 模拟：label=1 (好评), label=0 (差评)
data = [
    ["label", "text"],
    [1, "这个算法课程真的很棒，深入浅出！"],
    [0, "环境配置太麻烦了，报错一大堆，体验极差。"],
    [1, "AI是未来的趋势，必须掌握。"],
    [0, "听不懂，退钱！"],
    [1, "Data Pipeline的设计非常巧妙，解决了内存溢出问题。"]
]

with open(csv_path, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(data)

print(f"✅ 模拟数据已生成: {csv_path}")

✅ 模拟数据已生成: ../../data/raw_data/comments.csv


### 2.2 加载数据（Load Dataset）

In [ ]:
from datasets import load_dataset

# 工业界标准操作：从本地文件加载
# 这一步非常快，因为只是建立了映射，没有真正读入内存
dataset = load_dataset("csv", data_files=csv_path)

print(dataset)
# 输出: DatasetDict({ train: Dataset({ features: ['label', 'text'], num_rows: 5 }) })
# 你会发现它自动把 CSV 变成了类似字典的结构

/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 5 examples [00:00, 934.60 examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5
    })
})


### 2.3 核武器：`map` 函数 (The Mapper)

这是今天**最重要**的代码。我们要把 Tokenizer 应用到所有样本上。千万不要用 `for` 循环！要用 `.map()` 实现多进程并行处理。

In [ ]:
from transformers import BertTokenizer

# 加载本地 Tokenizer (复用之前的路径)
model_path = "../../data/pretrained_models/bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_path)

# --- 定义预处理函数 ---
def preprocess_function(examples):
    # examples['text'] 是一个 List，Tokenizer 可以直接处理 List
    # 注意：这里我们只做截断 (Truncation)，不做 Padding！
    # 为什么？(面试坑点，见下文 DataCollator 部分)
    return tokenizer(
        examples["text"], 
        truncation=True, 
        max_length=512
    )

# --- 一键映射 ---
# batched=True: 每次处理一批数据（默认1000条），利用 tokenizer 的并行能力
# num_proc=1: 如果数据量大，可以开多进程 (e.g., num_proc=4)
encoded_dataset = dataset.map(preprocess_function, batched=True)

print("处理后的列名:", encoded_dataset['train'].column_names)
# 输出应包含: input_ids, token_type_ids, attention_mask, label, text

Map: 100%|██████████| 5/5 [00:00<00:00, 543.42 examples/s]

处理后的列名: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


### 2.4 动态对齐：Data Collator (The Dynamic Padder)

**面试必考题**：*“为什么不在 `map` 阶段直接 `padding=True`？”*

  * **Static Padding (静态对齐)**: 如果在 map 阶段做，所有句子都会被补齐到 `max_length` (比如 512)。哪怕一个 Batch 里全是短句，也要补到 512。**浪费显存和计算时间**。
  * **Dynamic Padding (动态对齐)**: 我们把 Padding 推迟到 DataLoader 取出一个 Batch 的时候做。
      * 如果这个 Batch 最长只有 10 个字，那就大家都补到 10。
      * **Data Collator** 就是做这个脏活累活的。

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

# 1. 定义整理员
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- 🛠️ 修复核心：改名与格式设置 ---
# 1. 先把 'label' 改名为 'labels' (复数!)
# 这是为了配合 BERT 模型的 forward(labels=...) 接口
if 'label' in encoded_dataset['train'].column_names:
    encoded_data00set = encoded_dataset.rename_column("label", "labels")

# 2. 移除不需要的 'text' 列
final_dataset = encoded_dataset['train'].remove_columns(["text"])

# 3. 设置格式为 PyTorch Tensor
# DataCollator 只会处理 Tensor 类型的列，如果这里不转，label 可能会被丢弃
final_dataset.set_format("torch")

# 3. 创建 DataLoader
dataloader = DataLoader(
    final_dataset, 
    batch_size=2, 
    collate_fn=data_collator, 
    shuffle=True
)

# --- 验证流水线 ---
print("\n--- 模拟训练过程 ---")
for batch in dataloader:
    print("Batch Keys:", batch.keys()) # 此时你应该能看到 'labels'
    print("Input IDs Shape:", batch['input_ids'].shape)
    
    # --- 🛠️ 修复点：访问 'labels' (复数) ---
    print("Labels:", batch['labels']) 
    print("-" * 20)
    break


--- 模拟训练过程 ---
Batch Keys: KeysView({'labels': tensor([1, 0]), 'input_ids': tensor([[ 101,  100, 3221, 3313, 3341, 4638, 6633, 1232, 8024, 2553, 7557, 2958,
         2995,  511,  102,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 4384, 1862, 6981, 5390, 1922, 7937, 4172,  749, 8024, 2845, 7231,
          671, 1920, 1831, 8024,  860, 7741, 3353, 2345,  511,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])})
Input IDs Shape: torch.Size([2, 22])
Labels: tensor([1, 0])
--------------------


**预期结果**：
你会看到 `Input IDs Shape` 可能是 `[2, 15]` 或者 `[2, 28]`（取决于这个 Batch 里最长的那句有多长），而不是固定的 `[2, 512]`。**这就是效率！**

## 3 💥 算法岗面试深挖 (Interview Dive)

**Q1: 在处理大规模数据时，`map` 函数中的 `batched=True` 有什么作用？**

  * **A**: Tokenizer 内部是用 Rust 写的，对列表处理有极大的并行加速。开启 `batched=True` 一次性传给它 1000 条，比调 1000 次 Python 函数快几十倍。

**Q2: 什么是动态 Padding？有什么好处？**

  * **A**: 在组装 Batch 时，按照当前 Batch 的最大长度进行补齐，而不是全局最大长度。这能显著减少无效计算（Padding 的部分虽然被 Mask 了，但矩阵乘法还是要算的），提升训练速度约 30%-50%。


## 4 ⚔️ 每日算法题 (LeetCode Daily)

今天我们要练习一道与 **“数据分组/哈希”** 强相关的题目，这在大厂算法岗面试中非常常见。

### 🎯 题目: [LeetCode 49. 字母异位词分组 (Group Anagrams)](../../LeetCode%20practice/1-50.ipynb)

  * **难度**: Medium
  * **描述**: 给你一个字符串数组，请你将 **字母异位词** 组合在一起。
      * 输入: `["eat", "tea", "tan", "ate", "nat", "bat"]`
      * 输出: `[["bat"], ["nat","tan"], ["ate","eat","tea"]]`
  * **关联**: 这道题本质上就是 **Data Collator** 的逻辑——把特征相同（这里是字母组成相同）的数据归类到一起。
  * **核心思路**: 哈希表 (HashMap)。
      * Key 是什么？（排序后的字符串，或者字符计数元组）。
      * Value 是什么？（原字符串列表）。

## ✅ Day 4 任务清单

1.  **跑通代码**: 生成 CSV -\> Load -\> Map -\> Collate -\> DataLoader。确保你看懂了每一步的输入输出。
2.  **理解差异**: 彻底搞懂 **Static Padding (在 map 里做)** 和 **Dynamic Padding (在 collator 里做)** 的区别。
3.  **LeetCode**: 完成 LeetCode 49。

完成后，请回复 **“Day 4 Pipeline 搭建完毕”**。
明天（Day 5），我们将把“模型”和“数据”结合起来，进行 **Week 6 的综合复盘**，并为你即将到来的 **Week 7 (微调实战)** 做最后的准备！